# Определение пола человека по его имени
Работу выполнили студентки 2 курса Высшей школы ИТИС Грибанова Анастасия и Шафеева Дина, группа 11-803.

**Цель проекта:** научиться по имени, фамилии и отчеству человека определять его пол. 

Для достижения этой цели берутся данные 62609 человек и на их основе строятся различные модели.
Для работы берутся данные русскоязычного населения, из-за наличия у тех отчества, которое будет являться дополнительным параметром для анализа.

**Поля:** полями данных являются полные имена, а именно: фамилия, имя, отчество человека. Последнее поле в таблице - пол.

**Источник:** https://github.com/Rai220/MlSexDetector/blob/master/dataMini.csv

In [23]:
import chardet
import pandas as pd
from statistics import mode

with open('C:/Users/Nastya/dataMini.csv', 'rb') as f:
    result = chardet.detect(f.readline()) 


data = pd.read_csv('C:/Users/Nastya/dataMini.csv', encoding=result['encoding'])
data.head()

,ФАМИЛИЯ,ИМЯ,ОТЧЕСТВО,ПОЛ
0,АБАБКОВ,СЕРГЕЙ,ИВАНОВИЧ,М
1,АБАЕВ,АНДРЕЙ,АЛЕКСАНДРОВИЧ,М
2,АБАЗОВА,ЕЛЕНА,НИКОЛАЕВНА,Ж
3,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М
4,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М


In [24]:
data.shape

(62609, 4)

Узнаем, есть ли в данных пропущенные значения, с помощью функции isnull()

In [25]:
data.isnull().any().any()

True

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62609 entries, 0 to 62608
Data columns (total 4 columns):
ФАМИЛИЯ     62609 non-null object
ИМЯ         62609 non-null object
ОТЧЕСТВО    62579 non-null object
ПОЛ         62609 non-null object
dtypes: object(4)
memory usage: 1.9+ MB


Воспользовавшись функцией info(), выяснили, что пропуски встречаются только в столбце «Отчество», и их не так много, всего 30, так что можем заменить их на самое часто встречающееся значение в этом столбце - моду. Найдем это значение.

In [27]:
print(data.mode()['ОТЧЕСТВО'])

0    АЛЕКСАНДРОВНА
Name: ОТЧЕСТВО, dtype: object


Подставляем эту моду ('Александровна') в пропущенные ячейки.

In [28]:
data = data.fillna('АЛЕКСАНДРОВНА')
print(data)

         ФАМИЛИЯ     ИМЯ       ОТЧЕСТВО ПОЛ
0        АБАБКОВ  СЕРГЕЙ       ИВАНОВИЧ   М
1          АБАЕВ  АНДРЕЙ  АЛЕКСАНДРОВИЧ   М
2        АБАЗОВА   ЕЛЕНА     НИКОЛАЕВНА   Ж
3       АБАКУМОВ  СЕРГЕЙ     ВАСИЛЬЕВИЧ   М
4       АБАКУМОВ  СЕРГЕЙ     ВАСИЛЬЕВИЧ   М
...          ...     ...            ...  ..
62604     ЖАРОВА  МАРИНА    АНАТОЛЬЕВНА   Ж
62605  СИРОТКИНА  ГАЛИНА     НИКОЛАЕВНА   Ж
62606    ЧУВИЛИН  СЕРГЕЙ     ВИКТОРОВИЧ   М
62607    БЕКАСОВ  МИХАИЛ     НИКОЛАЕВИЧ   М
62608     ЛЕЗОВА    ИННА     НИКОЛАЕВНА   Ж

[62609 rows x 4 columns]


Еще раз проверим наши данные на наличие пропущенных значений с помощью той же функции isnull().

In [29]:
data.isnull().any().any()

False

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62609 entries, 0 to 62608
Data columns (total 4 columns):
ФАМИЛИЯ     62609 non-null object
ИМЯ         62609 non-null object
ОТЧЕСТВО    62609 non-null object
ПОЛ         62609 non-null object
dtypes: object(4)
memory usage: 1.9+ MB


Пропущенные данные успешно заполнились модой

   Наш алгоритм будет строиться на основе окончаний отчеств. Мы выяснили, что большинство отчеств оканчиваются на "ич", "лы", "на", "зы" и "ва". Из них первые два относятся к мужским отчествам, а последние к женским.

   Но при этом в выборке будут встречаться и отчества, которые не относятся к данным категориям, имеющие другие окончания.

   Чтобы узнать, к какому полу чаще всего относятся люди, имеющие такие отчества, мы прошлись по всем данным и посчитали сколько человек относится к мужскому полу, а сколько к женскому.

In [31]:
male = 0
female = 0
for index, row in data.iterrows():
    endOfLastname = row['ОТЧЕСТВО'][len(row['ОТЧЕСТВО'])-2:]
    if endOfLastname!='ИЧ' and endOfLastname!='ЛЫ' and endOfLastname!='НА' and endOfLastname!='ЗЫ' and endOfLastname!='ВА':
        if row['ПОЛ']=='М':
            male=male+1
        else:
            female=female+1
print("count of male = ", male, " count of female = ", female)

count of male =  195  count of female =  112


В результате мы выяснили, что мужчин 195, а женщин 112. Так как мужчин больше, мы решили причислять всех людей с отличными от нашей классификации окончаний отчествами к мужскому полу.

In [32]:
data['Pred'] = None
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62609 entries, 0 to 62608
Data columns (total 5 columns):
ФАМИЛИЯ     62609 non-null object
ИМЯ         62609 non-null object
ОТЧЕСТВО    62609 non-null object
ПОЛ         62609 non-null object
Pred        0 non-null object
dtypes: object(5)
memory usage: 2.4+ MB


Далее мы создали новый столбец для нашей модели, прошлись по всем данным и заполнили его.

In [33]:
for index, row in data.iterrows():
    endOfLastname = row['ОТЧЕСТВО'][len(row['ОТЧЕСТВО'])-2:]
    if endOfLastname=='НА' or endOfLastname=='ЗЫ' or endOfLastname=='ВА':
        row['Pred'] = 'Ж'
    else:
        row['Pred'] = 'М'
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62609 entries, 0 to 62608
Data columns (total 5 columns):
ФАМИЛИЯ     62609 non-null object
ИМЯ         62609 non-null object
ОТЧЕСТВО    62609 non-null object
ПОЛ         62609 non-null object
Pred        62609 non-null object
dtypes: object(5)
memory usage: 2.4+ MB


In [34]:
data.head()

,ФАМИЛИЯ,ИМЯ,ОТЧЕСТВО,ПОЛ,Pred
0,АБАБКОВ,СЕРГЕЙ,ИВАНОВИЧ,М,М
1,АБАЕВ,АНДРЕЙ,АЛЕКСАНДРОВИЧ,М,М
2,АБАЗОВА,ЕЛЕНА,НИКОЛАЕВНА,Ж,Ж
3,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М,М
4,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М,М


In [35]:
data.tail()

,ФАМИЛИЯ,ИМЯ,ОТЧЕСТВО,ПОЛ,Pred
62604,ЖАРОВА,МАРИНА,АНАТОЛЬЕВНА,Ж,Ж
62605,СИРОТКИНА,ГАЛИНА,НИКОЛАЕВНА,Ж,Ж
62606,ЧУВИЛИН,СЕРГЕЙ,ВИКТОРОВИЧ,М,М
62607,БЕКАСОВ,МИХАИЛ,НИКОЛАЕВИЧ,М,М
62608,ЛЕЗОВА,ИННА,НИКОЛАЕВНА,Ж,Ж


In [36]:
accuracy = 0
for index, row in data.iterrows():
    if row['Pred']==row['ПОЛ']:
        accuracy=accuracy+1
print('accuracy is ', accuracy/len(data))

accuracy is  0.9963583510357936


Чтобы выяснить точность нашей модели, мы еще раз прошлись по всем данным и сравнили готовое значение пола с нашим прогнозом. Результат поделили на количество всех элементов. 

Итак, точность нашего алгоритма равняется 0,996.

Для сравнения воспользуемся методами машинного обучения. Для этого еще раз считаем данные, чтобы избавиться от изменений, которые мы внесли при предыдущей работе с ними.

In [48]:
with open('C:/Users/Nastya/dataMini.csv', 'rb') as f:
    result = chardet.detect(f.readline()) 


data1 = pd.read_csv('C:/Users/Nastya/dataMini.csv', encoding=result['encoding'])
data1.head()

,ФАМИЛИЯ,ИМЯ,ОТЧЕСТВО,ПОЛ
0,АБАБКОВ,СЕРГЕЙ,ИВАНОВИЧ,М
1,АБАЕВ,АНДРЕЙ,АЛЕКСАНДРОВИЧ,М
2,АБАЗОВА,ЕЛЕНА,НИКОЛАЕВНА,Ж
3,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М
4,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М


Прежде всего опять заполним пропущенные значения в столбце "Отчество" на чаще встречающееся - Александровна.

In [49]:
data1 = data1.fillna('АЛЕКСАНДРОВНА')
print(data1)

         ФАМИЛИЯ     ИМЯ       ОТЧЕСТВО ПОЛ
0        АБАБКОВ  СЕРГЕЙ       ИВАНОВИЧ   М
1          АБАЕВ  АНДРЕЙ  АЛЕКСАНДРОВИЧ   М
2        АБАЗОВА   ЕЛЕНА     НИКОЛАЕВНА   Ж
3       АБАКУМОВ  СЕРГЕЙ     ВАСИЛЬЕВИЧ   М
4       АБАКУМОВ  СЕРГЕЙ     ВАСИЛЬЕВИЧ   М
...          ...     ...            ...  ..
62604     ЖАРОВА  МАРИНА    АНАТОЛЬЕВНА   Ж
62605  СИРОТКИНА  ГАЛИНА     НИКОЛАЕВНА   Ж
62606    ЧУВИЛИН  СЕРГЕЙ     ВИКТОРОВИЧ   М
62607    БЕКАСОВ  МИХАИЛ     НИКОЛАЕВИЧ   М
62608     ЛЕЗОВА    ИННА     НИКОЛАЕВНА   Ж

[62609 rows x 4 columns]


Так как большинство методов машинного обучения может работать только с числовыми значениями, нам нужно изменить наши строковые данные. Для анализа имен достаточно иметь информацию об окончаниях всех частей ФИО. Для имени и фамилии нам будет достаточно последней буквы, а для отчества возьмем последние две.

In [51]:
data1['ФАМ_ОКОНЧ'] = None
data1['ИМЯ_ОКОНЧ'] = None
data1['ОТЧ_ОКОНЧ'] = None
data1.head()

,ФАМИЛИЯ,ИМЯ,ОТЧЕСТВО,ПОЛ,ФАМИЛИЯ_КОД,ИМЯ_КОД,ОТЧЕСТВО_КОД,ФАМ_ОКОНЧ,ИМЯ_ОКОНЧ,ОТЧ_ОКОНЧ
0,АБАБКОВ,СЕРГЕЙ,ИВАНОВИЧ,М,None,None,None,None,None,None
1,АБАЕВ,АНДРЕЙ,АЛЕКСАНДРОВИЧ,М,None,None,None,None,None,None
2,АБАЗОВА,ЕЛЕНА,НИКОЛАЕВНА,Ж,None,None,None,None,None,None
3,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М,None,None,None,None,None,None
4,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М,None,None,None,None,None,None


In [52]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62609 entries, 0 to 62608
Data columns (total 10 columns):
ФАМИЛИЯ         62609 non-null object
ИМЯ             62609 non-null object
ОТЧЕСТВО        62609 non-null object
ПОЛ             62609 non-null object
ФАМИЛИЯ_КОД     0 non-null object
ИМЯ_КОД         0 non-null object
ОТЧЕСТВО_КОД    0 non-null object
ФАМ_ОКОНЧ       0 non-null object
ИМЯ_ОКОНЧ       0 non-null object
ОТЧ_ОКОНЧ       0 non-null object
dtypes: object(10)
memory usage: 4.8+ MB


In [53]:
for index, row in data1.iterrows():
    endOfSurname = row['ФАМИЛИЯ'][len(row['ФАМИЛИЯ'])-1]
    endOfName = row['ИМЯ'][len(row['ИМЯ'])-1]
    endOfLastname = row['ОТЧЕСТВО'][len(row['ОТЧЕСТВО'])-2:]
    row['ФАМ_ОКОНЧ'] = endOfSurname
    row['ИМЯ_ОКОНЧ'] = endOfName
    row['ОТЧ_ОКОНЧ'] = endOfLastname
data1.head()

,ФАМИЛИЯ,ИМЯ,ОТЧЕСТВО,ПОЛ,ФАМИЛИЯ_КОД,ИМЯ_КОД,ОТЧЕСТВО_КОД,ФАМ_ОКОНЧ,ИМЯ_ОКОНЧ,ОТЧ_ОКОНЧ
0,АБАБКОВ,СЕРГЕЙ,ИВАНОВИЧ,М,None,None,None,В,Й,ИЧ
1,АБАЕВ,АНДРЕЙ,АЛЕКСАНДРОВИЧ,М,None,None,None,В,Й,ИЧ
2,АБАЗОВА,ЕЛЕНА,НИКОЛАЕВНА,Ж,None,None,None,А,А,НА
3,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М,None,None,None,В,Й,ИЧ
4,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М,None,None,None,В,Й,ИЧ


Создадим словари для конвертирования окончаний в числа. При этом 1 будет значить принадлежность к женскому полу, -1 - к мужскому, а 0 - неопределенные данные. Заполним этими значениями новые столбцы для кодировки окончаний, а все пропущенные значения заменим на 0.

In [50]:
data1['ФАМИЛИЯ_КОД'] = None
data1['ИМЯ_КОД'] = None
data1['ОТЧЕСТВО_КОД'] = None
data1.head()

,ФАМИЛИЯ,ИМЯ,ОТЧЕСТВО,ПОЛ,ФАМИЛИЯ_КОД,ИМЯ_КОД,ОТЧЕСТВО_КОД
0,АБАБКОВ,СЕРГЕЙ,ИВАНОВИЧ,М,None,None,None
1,АБАЕВ,АНДРЕЙ,АЛЕКСАНДРОВИЧ,М,None,None,None
2,АБАЗОВА,ЕЛЕНА,НИКОЛАЕВНА,Ж,None,None,None
3,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М,None,None,None
4,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М,None,None,None


In [55]:
dct = {'А': 1, 'Я': 1, 'Н': -1, 'В': -1, 'Й': -1} 
data1['ФАМИЛИЯ_КОД'] = data1['ФАМ_ОКОНЧ'].map(dct)
data1.head()

,ФАМИЛИЯ,ИМЯ,ОТЧЕСТВО,ПОЛ,ФАМИЛИЯ_КОД,ИМЯ_КОД,ОТЧЕСТВО_КОД,ФАМ_ОКОНЧ,ИМЯ_ОКОНЧ,ОТЧ_ОКОНЧ
0,АБАБКОВ,СЕРГЕЙ,ИВАНОВИЧ,М,-1.0,None,None,В,Й,ИЧ
1,АБАЕВ,АНДРЕЙ,АЛЕКСАНДРОВИЧ,М,-1.0,None,None,В,Й,ИЧ
2,АБАЗОВА,ЕЛЕНА,НИКОЛАЕВНА,Ж,1.0,None,None,А,А,НА
3,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М,-1.0,None,None,В,Й,ИЧ
4,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М,-1.0,None,None,В,Й,ИЧ


In [57]:
dct_third_name = {'ИЧ':-1, 'ЛЫ':-1, 'НА':1, 'ЗЫ':1, 'ВА':1}
data1['ОТЧЕСТВО_КОД'] = data1['ОТЧ_ОКОНЧ'].map(dct_third_name)
data1.head()

,ФАМИЛИЯ,ИМЯ,ОТЧЕСТВО,ПОЛ,ФАМИЛИЯ_КОД,ИМЯ_КОД,ОТЧЕСТВО_КОД,ФАМ_ОКОНЧ,ИМЯ_ОКОНЧ,ОТЧ_ОКОНЧ
0,АБАБКОВ,СЕРГЕЙ,ИВАНОВИЧ,М,-1.0,None,-1.0,В,Й,ИЧ
1,АБАЕВ,АНДРЕЙ,АЛЕКСАНДРОВИЧ,М,-1.0,None,-1.0,В,Й,ИЧ
2,АБАЗОВА,ЕЛЕНА,НИКОЛАЕВНА,Ж,1.0,None,1.0,А,А,НА
3,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М,-1.0,None,-1.0,В,Й,ИЧ
4,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М,-1.0,None,-1.0,В,Й,ИЧ


In [62]:
dct_name = {'А':1, 'Я':1, 'М':-1, 'Й':-1, 'Н':-1, 'Т':-1, 'Р':-1}
data1['ИМЯ_КОД'] = data1['ИМЯ_ОКОНЧ'].map(dct_name)
data1.head()

,ФАМИЛИЯ,ИМЯ,ОТЧЕСТВО,ПОЛ,ФАМИЛИЯ_КОД,ИМЯ_КОД,ОТЧЕСТВО_КОД,ФАМ_ОКОНЧ,ИМЯ_ОКОНЧ,ОТЧ_ОКОНЧ
0,АБАБКОВ,СЕРГЕЙ,ИВАНОВИЧ,М,-1.0,-1.0,-1.0,В,Й,ИЧ
1,АБАЕВ,АНДРЕЙ,АЛЕКСАНДРОВИЧ,М,-1.0,-1.0,-1.0,В,Й,ИЧ
2,АБАЗОВА,ЕЛЕНА,НИКОЛАЕВНА,Ж,1.0,1.0,1.0,А,А,НА
3,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М,-1.0,-1.0,-1.0,В,Й,ИЧ
4,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М,-1.0,-1.0,-1.0,В,Й,ИЧ


In [63]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62609 entries, 0 to 62608
Data columns (total 10 columns):
ФАМИЛИЯ         62609 non-null object
ИМЯ             62609 non-null object
ОТЧЕСТВО        62609 non-null object
ПОЛ             62609 non-null object
ФАМИЛИЯ_КОД     55897 non-null float64
ИМЯ_КОД         56206 non-null float64
ОТЧЕСТВО_КОД    62302 non-null float64
ФАМ_ОКОНЧ       62609 non-null object
ИМЯ_ОКОНЧ       62609 non-null object
ОТЧ_ОКОНЧ       62609 non-null object
dtypes: float64(3), object(7)
memory usage: 4.8+ MB


In [65]:
data1 = data1.fillna(0)
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62609 entries, 0 to 62608
Data columns (total 10 columns):
ФАМИЛИЯ         62609 non-null object
ИМЯ             62609 non-null object
ОТЧЕСТВО        62609 non-null object
ПОЛ             62609 non-null object
ФАМИЛИЯ_КОД     62609 non-null float64
ИМЯ_КОД         62609 non-null float64
ОТЧЕСТВО_КОД    62609 non-null float64
ФАМ_ОКОНЧ       62609 non-null object
ИМЯ_ОКОНЧ       62609 non-null object
ОТЧ_ОКОНЧ       62609 non-null object
dtypes: float64(3), object(7)
memory usage: 4.8+ MB


Таким же образом закодируем пол: женский - 1, мужской - -1.

In [66]:
data1['ПОЛ_КОД'] = None

In [67]:
dct_gender = {'Ж':1, 'М':-1}
data1['ПОЛ_КОД'] = data1['ПОЛ'].map(dct_gender)
data1.head()

,ФАМИЛИЯ,ИМЯ,ОТЧЕСТВО,ПОЛ,ФАМИЛИЯ_КОД,ИМЯ_КОД,ОТЧЕСТВО_КОД,ФАМ_ОКОНЧ,ИМЯ_ОКОНЧ,ОТЧ_ОКОНЧ,ПОЛ_КОД
0,АБАБКОВ,СЕРГЕЙ,ИВАНОВИЧ,М,-1.0,-1.0,-1.0,В,Й,ИЧ,-1
1,АБАЕВ,АНДРЕЙ,АЛЕКСАНДРОВИЧ,М,-1.0,-1.0,-1.0,В,Й,ИЧ,-1
2,АБАЗОВА,ЕЛЕНА,НИКОЛАЕВНА,Ж,1.0,1.0,1.0,А,А,НА,1
3,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М,-1.0,-1.0,-1.0,В,Й,ИЧ,-1
4,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М,-1.0,-1.0,-1.0,В,Й,ИЧ,-1


Теперь можем удалить столбцы с окончаниями, так как они нам больше не нужны.

In [68]:
data1.drop(['ФАМ_ОКОНЧ', 'ИМЯ_ОКОНЧ', 'ОТЧ_ОКОНЧ'], axis='columns', inplace=True)
data1.head()

,ФАМИЛИЯ,ИМЯ,ОТЧЕСТВО,ПОЛ,ФАМИЛИЯ_КОД,ИМЯ_КОД,ОТЧЕСТВО_КОД,ПОЛ_КОД
0,АБАБКОВ,СЕРГЕЙ,ИВАНОВИЧ,М,-1.0,-1.0,-1.0,-1
1,АБАЕВ,АНДРЕЙ,АЛЕКСАНДРОВИЧ,М,-1.0,-1.0,-1.0,-1
2,АБАЗОВА,ЕЛЕНА,НИКОЛАЕВНА,Ж,1.0,1.0,1.0,1
3,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М,-1.0,-1.0,-1.0,-1
4,АБАКУМОВ,СЕРГЕЙ,ВАСИЛЬЕВИЧ,М,-1.0,-1.0,-1.0,-1


Построим дерево решений для нашей выборки.

Создадим массив признаков из столбцов: "ФАМИЛИЯ_КОД", "ИМЯ_КОД", "ОТЧЕСТВО_КОД" и массив классов из столбца "ПОЛ_КОД". Разделим выборку на тестовую и трейновую, отдавая тестовой 30 процентов данных.

In [76]:
X = data1[['ФАМИЛИЯ_КОД', 'ИМЯ_КОД', 'ОТЧЕСТВО_КОД']]
y = data1[['ПОЛ_КОД']]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Найдем наилучшие параметры для нашего обучающегося дерева и обучим наше дерево на них. 

In [77]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
gs = GridSearchCV(DecisionTreeClassifier(random_state=17),
                  param_grid = {'max_depth' : range(1, 6),
                                'max_features' : ['auto', 'log2', None],
                                'min_samples_leaf' : range(1, 6),
                                'min_samples_split' : range(2, 6),
                                'criterion' : ['gini', 'entropy'],
                                'presort' : ['auto', True, False],
                                'splitter' : ['best', 'random']},
                  cv=7, scoring='accuracy'
                  ).fit(X_train, y_train)

Наилучшие параметры найдены:

In [78]:
gs.best_params_

{'criterion': 'gini',
 'max_depth': 3,
 'max_features': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'presort': 'auto',
 'splitter': 'best'}

In [79]:
tree=DecisionTreeClassifier(criterion = 'gini', max_depth = 3, min_samples_leaf = 1, min_samples_split = 2, presort = 'auto', splitter = 'best')
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='auto',
                       random_state=None, splitter='best')

In [80]:
tree.score(X_train, y_train)

0.9974672568794779

В результате точность нашего дерева решений на трейновой выборке равна 0,997467. 

In [81]:
tree.fit(X_test, y_test)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='auto',
                       random_state=None, splitter='best')

In [82]:
tree.score(X_test, y_test)

0.99787041473673

А точность нашего дерева решений на тестовой выборке равна 0,99787.